# Deep Dive 04: 冷启动场景下的 Uplift 建模

## 🎯 学习目标

完成本 notebook 后，你将能够：

1. **理解** 冷启动场景下 Uplift 建模的挑战
2. **掌握** Thompson Sampling 在 Uplift 中的应用
3. **实现** 带有先验更新的在线 Uplift 策略
4. **设计** 从探索到利用的平滑过渡机制

---

## 📖 背景故事

> **场景：新用户首单优惠策略**
>
> 你是电商平台的算法工程师，负责新用户的首单优惠策略。问题来了：
>
> - 新用户没有历史购买数据
> - 传统 Uplift 模型需要用户特征来预测处理效应
> - 对新用户，我们只有注册时的基础信息（年龄、城市、渠道）
>
> 产品经理问：**"对于一个刚注册的用户，我们应该发什么优惠券？"**
>
> **本 notebook 将解决这个冷启动 Uplift 问题！**

---

## Part 1: 冷启动问题分析

### 1.1 为什么冷启动是 Uplift 的难题？

| 传统 Uplift | 冷启动场景 |
|------------|----------|
| 丰富的用户历史特征 | 只有注册时的基础信息 |
| 可以训练精确的 CATE 模型 | 模型预测不可靠 |
| 直接用模型预测选择处理 | 需要探索与利用的权衡 |

### 1.2 核心挑战

1. **信息不足**：新用户缺乏行为数据，无法准确预测其对处理的响应
2. **探索-利用权衡**：是用当前最佳策略（利用）还是尝试新策略收集数据（探索）？
3. **冷启动速度**：需要快速积累足够数据来做出可靠决策

### 1.3 解决思路

```
传统 Uplift:  用户特征 → CATE 模型 → 选择处理

冷启动 Uplift: 
  ┌─────────────────────────────────────────┐
  │ 1. 设置合理先验 (基于群体统计)          │
  │ 2. Thompson Sampling 选择处理          │
  │ 3. 观察结果，更新后验                   │
  │ 4. 随着数据积累，逐渐收敛到最优策略     │
  └─────────────────────────────────────────┘
```

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.special import expit  # sigmoid
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = ['Arial Unicode MS', 'SimHei', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 环境准备完成")

---

## Part 2: Thompson Sampling 原理

### 2.1 贝叶斯方法回顾

对于二元结果（转化/不转化），我们使用 Beta-Bernoulli 模型：

- **先验**: $\theta \sim \text{Beta}(\alpha, \beta)$
- **似然**: $Y | \theta \sim \text{Bernoulli}(\theta)$
- **后验**: $\theta | Y \sim \text{Beta}(\alpha + \sum Y, \beta + n - \sum Y)$

### 2.2 Thompson Sampling 算法

```
对于每个用户:
  1. 对每种处理 k，从后验 Beta(α_k, β_k) 采样 θ̂_k
  2. 选择 θ̂ 最大的处理
  3. 观察结果 Y
  4. 更新后验：α_k += Y, β_k += (1-Y)
```

**关键洞见**：Thompson Sampling 自然地平衡探索与利用！
- 不确定性高的处理 → 采样方差大 → 有机会被选中（探索）
- 效果好的处理 → 后验均值高 → 更可能被选中（利用）

In [ ]:
# 可视化 Thompson Sampling 的探索-利用权衡

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 模拟不同阶段的后验分布
stages = [
    {'name': '初始阶段 (n=0)', 'params': [(1, 1), (1, 1), (1, 1)]},
    {'name': '探索阶段 (n=20)', 'params': [(5, 17), (8, 14), (6, 16)]},
    {'name': '收敛阶段 (n=100)', 'params': [(25, 77), (45, 57), (30, 72)]},
]

treatment_names = ['无优惠', '5折券', '满减券']
colors = ['#95a5a6', '#e74c3c', '#3498db']

for col, stage in enumerate(stages):
    ax = axes[0, col]
    x = np.linspace(0, 1, 200)
    
    for i, (a, b) in enumerate(stage['params']):
        y = stats.beta.pdf(x, a, b)
        ax.plot(x, y, label=f"{treatment_names[i]} (α={a}, β={b})", 
               color=colors[i], linewidth=2)
        ax.fill_between(x, y, alpha=0.2, color=colors[i])
    
    ax.set_xlabel('转化率 θ')
    ax.set_ylabel('密度')
    ax.set_title(stage['name'])
    ax.legend(fontsize=8)

# 模拟采样过程
np.random.seed(123)
n_samples = 1000

for col, stage in enumerate(stages):
    ax = axes[1, col]
    
    samples = []
    for a, b in stage['params']:
        samples.append(np.random.beta(a, b, n_samples))
    
    # 每次采样选择的处理
    choices = np.argmax(samples, axis=0)
    choice_counts = np.bincount(choices, minlength=3)
    
    ax.bar(treatment_names, choice_counts / n_samples, color=colors)
    ax.set_ylabel('选择概率')
    ax.set_title(f'处理选择分布 ({stage["name"]})')
    
    # 标注选择概率
    for i, count in enumerate(choice_counts):
        ax.text(i, count/n_samples + 0.02, f'{count/n_samples:.1%}', 
               ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n观察：")
print("  初始阶段：三种处理被选择的概率接近均等（探索）")
print("  探索阶段：开始向效果好的处理倾斜")
print("  收敛阶段：集中在最佳处理（利用）")

---

## Part 3: 基础 Thompson Sampling Uplift

### 🧪 TODO 练习 1: 实现基础 Thompson Sampling

请实现一个基础的 Thompson Sampling 策略，用于选择最佳处理。

In [ ]:
class BasicThompsonSampling:
    """
    基础 Thompson Sampling
    使用 Beta-Bernoulli 模型
    """
    def __init__(self, n_treatments, prior_alpha=1, prior_beta=1):
        """
        Args:
            n_treatments: 处理数量
            prior_alpha: Beta 先验的 α 参数
            prior_beta: Beta 先验的 β 参数
        """
        self.n_treatments = n_treatments
        # 初始化每种处理的 α 和 β
        self.alphas = [prior_alpha] * n_treatments
        self.betas = [prior_beta] * n_treatments
        self.n_pulls = [0] * n_treatments  # 记录每种处理被选择的次数
    
    def select_treatment(self):
        """
        使用 Thompson Sampling 选择处理
        
        Returns:
            selected: 选择的处理索引
        """
        # 实现 Thompson Sampling
        # 1. 对每种处理，从 Beta(α, β) 采样
        samples = [np.random.beta(self.alphas[i], self.betas[i]) 
                  for i in range(self.n_treatments)]
        
        # 2. 选择采样值最大的处理
        selected = np.argmax(samples)
        
        return selected
    
    def update(self, treatment, outcome):
        """
        观察结果后更新后验
        
        Args:
            treatment: 使用的处理
            outcome: 结果 (0 或 1)
        """
        # 更新后验参数
        # α += outcome, β += (1 - outcome)
        self.alphas[treatment] += outcome
        self.betas[treatment] += (1 - outcome)
        self.n_pulls[treatment] += 1
    
    def get_posterior_stats(self):
        """获取后验统计量"""
        stats = []
        for i in range(self.n_treatments):
            mean = self.alphas[i] / (self.alphas[i] + self.betas[i])
            std = np.sqrt(self.alphas[i] * self.betas[i] / 
                         ((self.alphas[i] + self.betas[i])**2 * 
                          (self.alphas[i] + self.betas[i] + 1)))
            stats.append({
                'mean': mean, 
                'std': std, 
                'alpha': self.alphas[i], 
                'beta': self.betas[i],
                'n_pulls': self.n_pulls[i]
            })
        return stats

print("✅ BasicThompsonSampling 类定义完成（已完成 TODO 部分）")

### 📝 参考答案

In [ ]:
# ============================================================
# 参考答案：基础 Thompson Sampling 完整实现
# ============================================================

class BasicThompsonSampling:
    """
    基础 Thompson Sampling - 完整实现
    """
    def __init__(self, n_treatments, prior_alpha=1, prior_beta=1):
        self.n_treatments = n_treatments
        self.alphas = [prior_alpha] * n_treatments
        self.betas = [prior_beta] * n_treatments
        self.n_pulls = [0] * n_treatments  # 记录每种处理被选择的次数
    
    def select_treatment(self):
        """使用 Thompson Sampling 选择处理"""
        # 从每种处理的后验采样
        samples = [np.random.beta(self.alphas[i], self.betas[i]) 
                  for i in range(self.n_treatments)]
        # 选择采样值最大的
        return np.argmax(samples)
    
    def update(self, treatment, outcome):
        """更新后验"""
        self.alphas[treatment] += outcome
        self.betas[treatment] += (1 - outcome)
        self.n_pulls[treatment] += 1
    
    def get_posterior_stats(self):
        """获取后验统计量"""
        stats = []
        for i in range(self.n_treatments):
            mean = self.alphas[i] / (self.alphas[i] + self.betas[i])
            std = np.sqrt(self.alphas[i] * self.betas[i] / 
                         ((self.alphas[i] + self.betas[i])**2 * 
                          (self.alphas[i] + self.betas[i] + 1)))
            stats.append({
                'mean': mean, 
                'std': std, 
                'alpha': self.alphas[i], 
                'beta': self.betas[i],
                'n_pulls': self.n_pulls[i]
            })
        return stats

# 测试
ts = BasicThompsonSampling(n_treatments=3)
print("✅ BasicThompsonSampling 创建成功")
print(f"   初始后验: {ts.get_posterior_stats()}")

---

## Part 4: 模拟冷启动场景

### 4.1 创建模拟环境

In [ ]:
class ColdStartEnvironment:
    """
    冷启动模拟环境
    
    模拟新用户首单场景：
    - 用户只有基础特征（用户群组）
    - 不同群组对不同处理的响应不同
    """
    def __init__(self, n_groups=3, n_treatments=3, seed=42):
        np.random.seed(seed)
        
        self.n_groups = n_groups
        self.n_treatments = n_treatments
        
        # 真实的转化率矩阵 (group x treatment)
        # 每个群组对每种处理的真实响应率
        self.true_conversion_rates = np.array([
            # 无优惠  5折券  满减券
            [0.05,   0.15,   0.10],  # 群组0: 价格敏感型，对折扣响应大
            [0.10,   0.12,   0.18],  # 群组1: 凑单型，对满减响应大
            [0.15,   0.16,   0.14],  # 群组2: 品质型，对优惠不太敏感
        ])
        
        # 真实最优处理
        self.optimal_treatments = np.argmax(self.true_conversion_rates, axis=1)
        
        # 群组分布
        self.group_probs = [0.4, 0.35, 0.25]  # 群组0最多
    
    def generate_user(self):
        """生成一个新用户"""
        group = np.random.choice(self.n_groups, p=self.group_probs)
        return {'group': group}
    
    def get_outcome(self, user, treatment):
        """获取用户在某处理下的结果"""
        group = user['group']
        prob = self.true_conversion_rates[group, treatment]
        return np.random.binomial(1, prob)
    
    def get_optimal_treatment(self, user):
        """获取用户的真实最优处理"""
        return self.optimal_treatments[user['group']]
    
    def get_oracle_value(self):
        """计算 Oracle 策略的期望转化率"""
        value = 0
        for g, prob in enumerate(self.group_probs):
            optimal_t = self.optimal_treatments[g]
            value += prob * self.true_conversion_rates[g, optimal_t]
        return value

# 创建环境
env = ColdStartEnvironment()

print("环境设置：")
print(f"\n真实转化率矩阵 (行=群组, 列=处理):")
treatment_names = ['无优惠', '5折券', '满减券']
df_rates = pd.DataFrame(env.true_conversion_rates, 
                        columns=treatment_names,
                        index=[f'群组{i}' for i in range(3)])
print(df_rates.to_string())

print(f"\n每个群组的最优处理:")
for g in range(3):
    print(f"  群组{g}: {treatment_names[env.optimal_treatments[g]]}")

print(f"\nOracle 策略期望转化率: {env.get_oracle_value():.3f}")

### 4.2 比较不同策略

In [ ]:
def simulate_policy(env, policy, n_users=1000, seed=42):
    """
    模拟策略运行
    
    Args:
        env: 环境
        policy: 策略对象（需要有 select_treatment 和 update 方法）
        n_users: 用户数
        
    Returns:
        history: 运行历史
    """
    np.random.seed(seed)
    
    history = {
        'cumulative_conversions': [],
        'cumulative_regret': [],
        'treatments': [],
        'outcomes': [],
        'optimal_treatments': []
    }
    
    total_conversions = 0
    total_regret = 0
    
    for t in range(n_users):
        # 生成用户
        user = env.generate_user()
        
        # 选择处理
        if hasattr(policy, 'select_treatment_for_user'):
            treatment = policy.select_treatment_for_user(user)
        else:
            treatment = policy.select_treatment()
        
        # 获取结果
        outcome = env.get_outcome(user, treatment)
        
        # 更新策略
        if hasattr(policy, 'update_for_user'):
            policy.update_for_user(user, treatment, outcome)
        else:
            policy.update(treatment, outcome)
        
        # 计算遗憾
        optimal_t = env.get_optimal_treatment(user)
        optimal_rate = env.true_conversion_rates[user['group'], optimal_t]
        actual_rate = env.true_conversion_rates[user['group'], treatment]
        regret = optimal_rate - actual_rate
        
        # 记录
        total_conversions += outcome
        total_regret += regret
        
        history['cumulative_conversions'].append(total_conversions)
        history['cumulative_regret'].append(total_regret)
        history['treatments'].append(treatment)
        history['outcomes'].append(outcome)
        history['optimal_treatments'].append(optimal_t)
    
    return history

# 定义不同策略

class RandomPolicy:
    """随机策略"""
    def __init__(self, n_treatments):
        self.n_treatments = n_treatments
    
    def select_treatment(self):
        return np.random.randint(self.n_treatments)
    
    def update(self, treatment, outcome):
        pass

class GreedyPolicy:
    """贪婪策略：总是选择当前观测转化率最高的处理"""
    def __init__(self, n_treatments):
        self.n_treatments = n_treatments
        self.successes = [0] * n_treatments
        self.trials = [0] * n_treatments
    
    def select_treatment(self):
        # 确保每种处理至少尝试一次
        for i in range(self.n_treatments):
            if self.trials[i] == 0:
                return i
        
        # 选择观测转化率最高的
        rates = [self.successes[i] / self.trials[i] for i in range(self.n_treatments)]
        return np.argmax(rates)
    
    def update(self, treatment, outcome):
        self.successes[treatment] += outcome
        self.trials[treatment] += 1

class EpsilonGreedy:
    """ε-Greedy 策略"""
    def __init__(self, n_treatments, epsilon=0.1):
        self.n_treatments = n_treatments
        self.epsilon = epsilon
        self.successes = [0] * n_treatments
        self.trials = [0] * n_treatments
    
    def select_treatment(self):
        if np.random.random() < self.epsilon:
            return np.random.randint(self.n_treatments)
        
        # 确保每种处理至少尝试一次
        for i in range(self.n_treatments):
            if self.trials[i] == 0:
                return i
        
        rates = [self.successes[i] / self.trials[i] for i in range(self.n_treatments)]
        return np.argmax(rates)
    
    def update(self, treatment, outcome):
        self.successes[treatment] += outcome
        self.trials[treatment] += 1

print("✅ 策略定义完成")

In [ ]:
# 运行模拟
n_users = 2000

policies = {
    '随机策略': RandomPolicy(n_treatments=3),
    '贪婪策略': GreedyPolicy(n_treatments=3),
    'ε-Greedy (ε=0.1)': EpsilonGreedy(n_treatments=3, epsilon=0.1),
    'Thompson Sampling': BasicThompsonSampling(n_treatments=3),
}

results = {}
for name, policy in policies.items():
    print(f"运行 {name}...")
    results[name] = simulate_policy(env, policy, n_users=n_users)

print("\n✅ 模拟完成")

In [ ]:
# 可视化比较
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

colors = {'随机策略': '#95a5a6', '贪婪策略': '#e74c3c', 
          'ε-Greedy (ε=0.1)': '#f39c12', 'Thompson Sampling': '#2ecc71'}

# 累计转化
ax1 = axes[0]
for name, history in results.items():
    ax1.plot(history['cumulative_conversions'], label=name, color=colors[name], linewidth=2)
ax1.set_xlabel('用户数')
ax1.set_ylabel('累计转化')
ax1.set_title('累计转化数对比')
ax1.legend()

# 累计遗憾
ax2 = axes[1]
for name, history in results.items():
    ax2.plot(history['cumulative_regret'], label=name, color=colors[name], linewidth=2)
ax2.set_xlabel('用户数')
ax2.set_ylabel('累计遗憾')
ax2.set_title('累计遗憾对比')
ax2.legend()

# 最终统计
ax3 = axes[2]
names = list(results.keys())
final_conversions = [results[n]['cumulative_conversions'][-1] for n in names]
bars = ax3.bar(range(len(names)), final_conversions, color=[colors[n] for n in names])
ax3.set_xticks(range(len(names)))
ax3.set_xticklabels(names, rotation=45, ha='right')
ax3.set_ylabel('总转化数')
ax3.set_title(f'{n_users} 用户后的总转化')

# 添加数值标签
for bar, conv in zip(bars, final_conversions):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
            f'{conv}', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

# 汇总表格
print("\n" + "=" * 60)
print("策略对比汇总")
print("=" * 60)
print(f"\n{'策略':<20} {'总转化':<10} {'转化率':<12} {'累计遗憾':<12}")
print("-" * 54)
for name in names:
    conv = results[name]['cumulative_conversions'][-1]
    rate = conv / n_users
    regret = results[name]['cumulative_regret'][-1]
    print(f"{name:<20} {conv:<10} {rate:<12.2%} {regret:<12.2f}")

---

## Part 5: 上下文感知的 Thompson Sampling

基础 Thompson Sampling 忽略了用户特征。现在我们扩展到**上下文感知**版本，为不同用户群体分别维护后验。

In [ ]:
class ContextualThompsonSampling:
    """
    上下文感知的 Thompson Sampling
    
    为每个用户群组分别维护后验分布
    """
    def __init__(self, n_treatments, n_groups, prior_alpha=1, prior_beta=1):
        self.n_treatments = n_treatments
        self.n_groups = n_groups
        
        # 为每个 (群组, 处理) 组合维护后验参数
        self.alphas = np.ones((n_groups, n_treatments)) * prior_alpha
        self.betas = np.ones((n_groups, n_treatments)) * prior_beta
        self.n_pulls = np.zeros((n_groups, n_treatments))
    
    def select_treatment_for_user(self, user):
        """根据用户群组选择处理"""
        group = user['group']
        
        # 从该群组的后验采样
        samples = [np.random.beta(self.alphas[group, t], self.betas[group, t]) 
                  for t in range(self.n_treatments)]
        
        return np.argmax(samples)
    
    def update_for_user(self, user, treatment, outcome):
        """更新特定群组的后验"""
        group = user['group']
        self.alphas[group, treatment] += outcome
        self.betas[group, treatment] += (1 - outcome)
        self.n_pulls[group, treatment] += 1
    
    def get_posterior_stats(self):
        """获取所有群组的后验统计量"""
        stats = {}
        for g in range(self.n_groups):
            stats[f'群组{g}'] = []
            for t in range(self.n_treatments):
                mean = self.alphas[g, t] / (self.alphas[g, t] + self.betas[g, t])
                stats[f'群组{g}'].append({
                    'mean': mean,
                    'alpha': self.alphas[g, t],
                    'beta': self.betas[g, t],
                    'n_pulls': self.n_pulls[g, t]
                })
        return stats

# 运行 Contextual Thompson Sampling
contextual_ts = ContextualThompsonSampling(n_treatments=3, n_groups=3)
results['Contextual TS'] = simulate_policy(env, contextual_ts, n_users=n_users)

print("\nContextual Thompson Sampling 后验统计：")
stats = contextual_ts.get_posterior_stats()
for group, group_stats in stats.items():
    print(f"\n{group}:")
    for t, t_stats in enumerate(group_stats):
        true_rate = env.true_conversion_rates[int(group[-1]), t]
        print(f"  {treatment_names[t]}: 估计={t_stats['mean']:.3f}, "
              f"真实={true_rate:.3f}, n={t_stats['n_pulls']:.0f}")

In [ ]:
# 对比基础 TS 和 Contextual TS
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 累计遗憾对比
ax1 = axes[0]
ax1.plot(results['Thompson Sampling']['cumulative_regret'], 
         label='基础 TS', linewidth=2, color='#3498db')
ax1.plot(results['Contextual TS']['cumulative_regret'], 
         label='Contextual TS', linewidth=2, color='#2ecc71')
ax1.set_xlabel('用户数')
ax1.set_ylabel('累计遗憾')
ax1.set_title('累计遗憾对比')
ax1.legend()

# 最优处理选择准确率
ax2 = axes[1]

# 计算滚动准确率
window = 100
for name in ['Thompson Sampling', 'Contextual TS']:
    history = results[name]
    correct = [1 if t == o else 0 
               for t, o in zip(history['treatments'], history['optimal_treatments'])]
    rolling_acc = pd.Series(correct).rolling(window=window).mean()
    ax2.plot(rolling_acc, label=name, linewidth=2)

ax2.set_xlabel('用户数')
ax2.set_ylabel('最优处理选择准确率 (滚动平均)')
ax2.set_title(f'最优处理选择准确率 (窗口={window})')
ax2.legend()

plt.tight_layout()
plt.show()

# 最终对比
print("\n" + "=" * 50)
print("基础 TS vs Contextual TS 对比")
print("=" * 50)

for name in ['Thompson Sampling', 'Contextual TS']:
    history = results[name]
    conv = history['cumulative_conversions'][-1]
    regret = history['cumulative_regret'][-1]
    acc = np.mean([1 if t == o else 0 
                   for t, o in zip(history['treatments'], history['optimal_treatments'])])
    print(f"\n{name}:")
    print(f"  总转化: {conv}")
    print(f"  累计遗憾: {regret:.2f}")
    print(f"  最优选择准确率: {acc:.2%}")

---

## Part 6: 带信息先验的 Thompson Sampling

在实际业务中，我们通常有历史数据可以设置更好的先验。

In [ ]:
class InformedThompsonSampling:
    """
    带信息先验的 Thompson Sampling
    
    使用历史数据设置先验
    """
    def __init__(self, n_treatments, n_groups, historical_data=None, 
                 prior_strength=10):
        """
        Args:
            historical_data: dict, 历史数据 {(group, treatment): (successes, trials)}
            prior_strength: 先验强度（相当于多少虚拟样本）
        """
        self.n_treatments = n_treatments
        self.n_groups = n_groups
        
        # 初始化先验
        self.alphas = np.ones((n_groups, n_treatments))
        self.betas = np.ones((n_groups, n_treatments))
        
        if historical_data is not None:
            for (g, t), (successes, trials) in historical_data.items():
                if trials > 0:
                    rate = successes / trials
                    # 将历史数据转化为等效的先验样本
                    self.alphas[g, t] = rate * prior_strength + 1
                    self.betas[g, t] = (1 - rate) * prior_strength + 1
        
        self.n_pulls = np.zeros((n_groups, n_treatments))
    
    def select_treatment_for_user(self, user):
        group = user['group']
        samples = [np.random.beta(self.alphas[group, t], self.betas[group, t]) 
                  for t in range(self.n_treatments)]
        return np.argmax(samples)
    
    def update_for_user(self, user, treatment, outcome):
        group = user['group']
        self.alphas[group, treatment] += outcome
        self.betas[group, treatment] += (1 - outcome)
        self.n_pulls[group, treatment] += 1

# 模拟历史数据（假设之前做过小规模测试）
historical_data = {
    # (群组, 处理): (成功次数, 总次数)
    (0, 0): (5, 100),   # 群组0 无优惠
    (0, 1): (14, 100),  # 群组0 5折券
    (0, 2): (9, 100),   # 群组0 满减券
    (1, 0): (10, 100),
    (1, 1): (11, 100),
    (1, 2): (17, 100),
    (2, 0): (15, 100),
    (2, 1): (16, 100),
    (2, 2): (13, 100),
}

print("历史数据设置的先验：")
print("(基于之前每组 100 样本的测试)")
print("\n" + "=" * 50)

for g in range(3):
    print(f"\n群组{g}:")
    for t in range(3):
        successes, trials = historical_data[(g, t)]
        rate = successes / trials
        true_rate = env.true_conversion_rates[g, t]
        print(f"  {treatment_names[t]}: 历史率={rate:.2f}, 真实率={true_rate:.2f}")

In [ ]:
# 对比无先验 vs 有先验
np.random.seed(42)

# 无信息先验
contextual_ts_flat = ContextualThompsonSampling(n_treatments=3, n_groups=3)
results_flat = simulate_policy(env, contextual_ts_flat, n_users=n_users, seed=42)

# 有信息先验
informed_ts = InformedThompsonSampling(n_treatments=3, n_groups=3, 
                                        historical_data=historical_data,
                                        prior_strength=50)
results_informed = simulate_policy(env, informed_ts, n_users=n_users, seed=42)

# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(results_flat['cumulative_regret'], label='无信息先验', linewidth=2)
ax1.plot(results_informed['cumulative_regret'], label='有信息先验', linewidth=2)
ax1.set_xlabel('用户数')
ax1.set_ylabel('累计遗憾')
ax1.set_title('累计遗憾对比')
ax1.legend()

# 前 500 用户放大
ax2 = axes[1]
ax2.plot(results_flat['cumulative_regret'][:500], label='无信息先验', linewidth=2)
ax2.plot(results_informed['cumulative_regret'][:500], label='有信息先验', linewidth=2)
ax2.set_xlabel('用户数')
ax2.set_ylabel('累计遗憾')
ax2.set_title('前 500 用户的累计遗憾（冷启动阶段）')
ax2.legend()

plt.tight_layout()
plt.show()

print("\n关键发现：")
print(f"  无信息先验 - 最终遗憾: {results_flat['cumulative_regret'][-1]:.2f}")
print(f"  有信息先验 - 最终遗憾: {results_informed['cumulative_regret'][-1]:.2f}")
print(f"  ✅ 有信息先验显著降低了冷启动阶段的遗憾！")

---

## Part 7: 实际部署考虑

### 7.1 分阶段策略

In [ ]:
class StagedUpliftPolicy:
    """
    分阶段 Uplift 策略
    
    阶段 1 (探索期): Thompson Sampling
    阶段 2 (利用期): 固定最优策略
    """
    def __init__(self, n_treatments, n_groups, exploration_users=500, 
                 min_samples_per_arm=50):
        self.n_treatments = n_treatments
        self.n_groups = n_groups
        self.exploration_users = exploration_users
        self.min_samples_per_arm = min_samples_per_arm
        
        self.ts = ContextualThompsonSampling(n_treatments, n_groups)
        self.n_users = 0
        self.best_treatments = None
        self.stage = 'exploration'
    
    def _determine_best_treatments(self):
        """确定每个群组的最优处理"""
        best = []
        for g in range(self.n_groups):
            # 检查样本量是否足够
            if np.min(self.ts.n_pulls[g]) < self.min_samples_per_arm:
                best.append(None)  # 样本不足，继续探索
            else:
                # 选择后验均值最高的处理
                means = self.ts.alphas[g] / (self.ts.alphas[g] + self.ts.betas[g])
                best.append(np.argmax(means))
        return best
    
    def select_treatment_for_user(self, user):
        group = user['group']
        
        if self.stage == 'exploration':
            return self.ts.select_treatment_for_user(user)
        else:
            if self.best_treatments[group] is not None:
                return self.best_treatments[group]
            else:
                # 该群组仍需探索
                return self.ts.select_treatment_for_user(user)
    
    def update_for_user(self, user, treatment, outcome):
        self.ts.update_for_user(user, treatment, outcome)
        self.n_users += 1
        
        # 检查是否进入利用期
        if self.n_users >= self.exploration_users and self.stage == 'exploration':
            self.best_treatments = self._determine_best_treatments()
            if all(t is not None for t in self.best_treatments):
                self.stage = 'exploitation'
                print(f"\n[用户 {self.n_users}] 进入利用期")
                print(f"  确定的最优策略: {[treatment_names[t] for t in self.best_treatments]}")

# 运行分阶段策略
staged_policy = StagedUpliftPolicy(n_treatments=3, n_groups=3, 
                                    exploration_users=500)
results['分阶段策略'] = simulate_policy(env, staged_policy, n_users=n_users, seed=42)

In [ ]:
# 最终对比
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

policies_to_compare = ['随机策略', 'Thompson Sampling', 'Contextual TS', '分阶段策略']
colors_compare = ['#95a5a6', '#3498db', '#2ecc71', '#9b59b6']

for name, color in zip(policies_to_compare, colors_compare):
    ax.plot(results[name]['cumulative_regret'], label=name, linewidth=2, color=color)

ax.axvline(x=500, color='gray', linestyle='--', alpha=0.5, label='探索期结束')
ax.set_xlabel('用户数')
ax.set_ylabel('累计遗憾')
ax.set_title('各策略累计遗憾对比')
ax.legend()

plt.tight_layout()
plt.show()

# 最终表格
print("\n" + "=" * 70)
print("最终结果汇总")
print("=" * 70)
print(f"\n{'策略':<20} {'总转化':<10} {'转化率':<12} {'累计遗憾':<12} {'vs 随机提升':<12}")
print("-" * 66)

random_conv = results['随机策略']['cumulative_conversions'][-1]

for name in policies_to_compare:
    history = results[name]
    conv = history['cumulative_conversions'][-1]
    rate = conv / n_users
    regret = history['cumulative_regret'][-1]
    improvement = (conv - random_conv) / random_conv
    print(f"{name:<20} {conv:<10} {rate:<12.2%} {regret:<12.2f} {improvement:<12.1%}")

---

## 💡 思考题

1. **非平稳环境**：如果用户偏好随时间变化，如何修改 Thompson Sampling？
   - 提示：考虑衰减历史数据的权重

2. **连续处理**：如果处理是连续的（如优惠金额），如何扩展？
   - 提示：考虑 Linear UCB 或 Gaussian Process

3. **延迟反馈**：如果转化有延迟（如 7 天后才知道是否复购），如何处理？
   - 提示：考虑 importance sampling 或 time-to-event 建模

4. **多目标**：如果既要考虑转化率又要考虑 LTV，如何平衡？
   - 提示：考虑复合指标或 Pareto 优化

---

## 📝 总结

### 核心方法

| 方法 | 适用场景 | 优点 | 缺点 |
|------|---------|------|------|
| 基础 TS | 全局最优处理 | 简单 | 忽略异质性 |
| Contextual TS | 群组差异 | 捕捉异质性 | 需要更多样本 |
| Informed TS | 有历史数据 | 快速收敛 | 依赖先验质量 |
| 分阶段策略 | 生产环境 | 平衡探索/利用 | 需要调参 |

### 实施建议

1. **冷启动阶段**：使用 Thompson Sampling + 信息先验
2. **数据积累后**：切换到 Contextual 版本
3. **足够数据后**：可以训练传统 Uplift 模型

### 面试要点

- 能解释 Thompson Sampling 的探索-利用权衡
- 理解 Beta-Bernoulli 共轭
- 能设计分阶段的策略
- 知道如何利用先验信息加速收敛